# Regression Week 2: Multiple Regression (Interpretation)

The goal of this first notebook is to explore multiple regression and feature engineering with existing graphlab functions.

In this notebook you will use data on house sales in King County to predict prices using multiple regression. You will:
* Use SFrames to do some feature engineering
* Use built-in graphlab functions to compute the regression weights (coefficients/parameters)
* Given the regression weights, predictors and outcome write a function to compute the Residual Sum of Squares
* Look at coefficients and interpret their meanings
* Evaluate multiple models via RSS

# Fire up graphlab create

In [1]:
import graphlab as gl

In [3]:
gl.product_key.get_product_key()

'88'

# Load in house sales data

Dataset is from house sales in King County, the region where the city of Seattle, WA is located.

In [4]:
sales = gl.SFrame('kc_house_data.gl/')

[INFO] graphlab.cython.cy_server: GraphLab Create v2.1 started. Logging: /tmp/graphlab_server_1491965328.log
INFO:graphlab.cython.cy_server:GraphLab Create v2.1 started. Logging: /tmp/graphlab_server_1491965328.log


This non-commercial license of GraphLab Create for academic use is assigned to  and .


In [5]:
sales.head(5)

id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront
7129300520,2014-10-13 00:00:00+00:00,221900.0,3.0,1.0,1180.0,5650,1,0
6414100192,2014-12-09 00:00:00+00:00,538000.0,3.0,2.25,2570.0,7242,2,0
5631500400,2015-02-25 00:00:00+00:00,180000.0,2.0,1.0,770.0,10000,1,0
2487200875,2014-12-09 00:00:00+00:00,604000.0,4.0,3.0,1960.0,5000,1,0
1954400510,2015-02-18 00:00:00+00:00,510000.0,3.0,2.0,1680.0,8080,1,0
view,condition,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat
0,3,7,1180,0,1955,0,98178,47.51123398
0,3,7,2170,400,1951,1991,98125,47.72102274
0,3,6,770,0,1933,0,98028,47.73792661
0,5,7,1050,910,1965,0,98136,47.52082


# Split data into training and testing.
We use seed=0 so that everyone running this notebook gets the same results.  In practice, you may set a random seed (or let GraphLab Create pick a random seed for you).  

In [7]:
train_data,test_data = sales.random_split(.8,seed=0)

# Learning a multiple regression model

Recall we can use the following code to learn a multiple regression model predicting 'price' based on the following features:
example_features = ['sqft_living', 'bedrooms', 'bathrooms'] on training data with the following code:

(Aside: We set validation_set = None to ensure that the results are always the same)

In [9]:
example_features = ['sqft_living', 'bedrooms', 'bathrooms']
example_model = gl.linear_regression.create(train_data, target = 'price', features = example_features, 
                                                  validation_set = None)

Linear regression:

--------------------------------------------------------

Number of examples          : 17384

Number of features          : 3

Number of unpacked features : 3

...

| Iteration | Passes   | Elapsed Time | Training-max_error | Training-rmse |

+-----------+----------+--------------+--------------------+---------------+

| 1         | 2        | 1.011581     | 4146407.600631     | 258679.804477 |

+-----------+----------+--------------+--------------------+---------------+

SUCCESS: Optimal solution found.

Now that we have fitted the model we can extract the regression weights (coefficients) as an SFrame as follows:

In [10]:
example_weight_summary = example_model.get("coefficients")
print example_weight_summary

+-------------+-------+----------------+---------------+
|     name    | index |     value      |     stderr    |
+-------------+-------+----------------+---------------+
| (intercept) |  None | 87910.0724924  |  7873.3381434 |
| sqft_living |  None | 315.403440552  | 3.45570032585 |
|   bedrooms  |  None | -65080.2155528 | 2717.45685442 |
|  bathrooms  |  None | 6944.02019265  | 3923.11493144 |
+-------------+-------+----------------+---------------+
[4 rows x 4 columns]



# Making Predictions

In the gradient descent notebook we use numpy to do our regression. In this book we will use existing graphlab create functions to analyze multiple regressions. 

Recall that once a model is built we can use the .predict() function to find the predicted values for data we pass. For example using the example model above:

In [11]:
example_predictions = example_model.predict(train_data)
print example_predictions[0] # should be 271789.505878

271789.505878


In [12]:
print type(example_predictions)

<class 'graphlab.data_structures.sarray.SArray'>


In [13]:
example_predictions

dtype: float
Rows: 17384
[271789.50587802153, 718880.3134862405, 207554.3108044818, 466612.0143411314, 436435.24634671584, 1568323.9489403444, 449210.3718142011, 237413.1031080943, 461031.5702092772, 506141.97895897925, 1032865.7246809743, 330561.6526197981, 336924.17472740513, 347277.2625128513, 502669.96886265534, 339178.7353570822, 343177.79024188174, 293867.746716668, 350694.81931260996, 873745.9750475689, 495834.85526313796, 305647.35306643357, 681031.9006199894, 677559.8905236654, 341178.262799482, 384234.69081021904, 655536.1029815841, 343495.76593268476, 574631.2621045958, 496416.35334817873, 635457.3895853374, 358102.9417902077, 585829.3703693207, 219325.42794728692, 655536.1029815841, 915480.4190428229, 394796.84769333777, 303329.84993323084, 906704.3484601371, 211862.85217312392, 567223.1396269982, 299075.76186115394, 778335.1481554355, 274943.5402835425, 773389.8829489796, 349540.31234948884, 594400.4890170001, 639184.4328689387, 737540.9975251283, 673305.8024515884, 810138

# Compute RSS

Now that we can make predictions given the model, let's write a function to compute the RSS of the model. Complete the function below to calculate RSS given the model, data, and the outcome.

In [15]:
def get_residual_sum_of_squares(model, data, outcome):
    # First get the predictions
    predicted_values=model.predict(data)
    # Then compute the residuals/errors
    residulas=outcome-predicted_values
    # Then square and add them up
    RSS=(residulas*residulas).sum()
    return(RSS)    

Test your function by computing the RSS on TEST data for the example model:

In [16]:
rss_example_train = get_residual_sum_of_squares(example_model, test_data, test_data['price'])
print rss_example_train # should be 2.7376153833e+14

2.7376153833e+14


# Create some new features

Although we often think of multiple regression as including multiple different features (e.g. # of bedrooms, squarefeet, and # of bathrooms) but we can also consider transformations of existing features e.g. the log of the squarefeet or even "interaction" features such as the product of bedrooms and bathrooms.

You will use the logarithm function to create a new feature. so first you should import it from the math library.

In [17]:
from math import log

Next create the following 4 new features as column in both TEST and TRAIN data:
* bedrooms_squared = bedrooms\*bedrooms
* bed_bath_rooms = bedrooms\*bathrooms
* log_sqft_living = log(sqft_living)
* lat_plus_long = lat + long 
As an example here's the first one:

In [18]:
train_data['bedrooms_squared'] = train_data['bedrooms'].apply(lambda x: x**2)
test_data['bedrooms_squared'] = test_data['bedrooms'].apply(lambda x: x**2)

In [19]:
type(train_data['bedrooms'])

graphlab.data_structures.sarray.SArray

In [28]:
print log(10,10)

1.0


In [29]:
print log(10)

2.30258509299


In [32]:
# create the remaining 3 features in both TEST and TRAIN data
train_data['bed_bath_rooms'] = train_data['bedrooms']*train_data['bathrooms']
test_data['bed_bath_rooms'] = test_data['bedrooms']*test_data['bathrooms']

train_data['log_sqft_living'] = train_data['sqft_living'].apply(lambda x:log(x))
test_data['log_sqft_living'] = test_data['sqft_living'].apply(lambda x:log(x))

train_data['lat_plus_long']=train_data['lat']+train_data['long']
test_data['lat_plus_long']=test_data['lat']+test_data['long']

* Squaring bedrooms will increase the separation between not many bedrooms (e.g. 1) and lots of bedrooms (e.g. 4) since 1^2 = 1 but 4^2 = 16. Consequently this feature will mostly affect houses with many bedrooms.
* bedrooms times bathrooms gives what's called an "interaction" feature. It is large when *both* of them are large.
* Taking the log of squarefeet has the effect of bringing large values closer together and spreading out small values.
* Adding latitude to longitude is totally non-sensical but we will do it anyway (you'll see why)

**Quiz Question: What is the mean (arithmetic average) value of your 4 new features on TEST data? (round to 2 digits)**

In [37]:
print len(test_data['bed_bath_rooms'])
print len(test_data['log_sqft_living'])
print len(test_data['lat_plus_long'])

print test_data['bed_bath_rooms'].size()
print test_data['log_sqft_living'].size()
print test_data['lat_plus_long'].size()

4229
4229
4229
4229
4229
4229


In [54]:
avg_bedrooms_squared = (test_data['bedrooms_squared'].sum())/len(test_data['bedrooms_squared'])
avg_bed_bath_rooms = (test_data['bed_bath_rooms'].sum())/len(test_data['bed_bath_rooms'])
avg_log_sqft_living = (test_data['log_sqft_living'].sum())/len(test_data['log_sqft_living'])
avg_lat_plus_long = (test_data['lat_plus_long'].sum())/len(test_data['lat_plus_long'])
print avg_bedrooms_squared, avg_bed_bath_rooms, avg_log_sqft_living, avg_lat_plus_long
print round(avg_bedrooms_squared,2), round(avg_bed_bath_rooms,2), round(avg_log_sqft_living,2), round(avg_lat_plus_long,2)

12.4466777016 7.50390163159 7.55027467965 -74.6533349722
12.45 7.5 7.55 -74.65


# Learning Multiple Models

Now we will learn the weights for three (nested) models for predicting house prices. The first model will have the fewest features the second model will add one more feature and the third will add a few more:
* Model 1: squarefeet, # bedrooms, # bathrooms, latitude & longitude
* Model 2: add bedrooms\*bathrooms
* Model 3: Add log squarefeet, bedrooms squared, and the (nonsensical) latitude + longitude

In [40]:
model_1_features = ['sqft_living', 'bedrooms', 'bathrooms', 'lat', 'long']
model_2_features = model_1_features + ['bed_bath_rooms']
model_3_features = model_2_features + ['bedrooms_squared', 'log_sqft_living', 'lat_plus_long']

Now that you have the features, learn the weights for the three different models for predicting target = 'price' using graphlab.linear_regression.create() and look at the value of the weights/coefficients:

In [42]:
# Learn the three models: (don't forget to set validation_set = None)
model_1_features_model= gl.linear_regression.create(train_data, target='price', features=model_1_features, validation_set=None)
model_2_features_model= gl.linear_regression.create(train_data, target='price', features=model_2_features, validation_set=None)
model_3_features_model= gl.linear_regression.create(train_data, target='price', features=model_3_features, validation_set=None)

Linear regression:

--------------------------------------------------------

Number of examples          : 17384

Number of features          : 5

Number of unpacked features : 5

...

| Iteration | Passes   | Elapsed Time | Training-max_error | Training-rmse |

+-----------+----------+--------------+--------------------+---------------+

| 1         | 2        | 0.026480     | 3193229.177891     | 228200.043154 |

+-----------+----------+--------------+--------------------+---------------+

SUCCESS: Optimal solution found.

In [45]:
# Examine/extract each model's coefficients:
model_1_features_model.get('coefficients')

name,index,value,stderr
(intercept),None,-56140675.759,1649985.42057
sqft_living,None,310.263325779,3.18882960407
bedrooms,None,-59577.1160679,2487.27977321
bathrooms,None,13811.840544,3593.54213295
lat,None,629865.78954,13120.7100325
long,None,-214790.285284,13284.2851627


In [46]:
model_2_features_model.get('coefficients')

name,index,value,stderr
(intercept),None,-54410676.1295,1650405.1657
sqft_living,None,304.449298058,3.20217535636
bedrooms,None,-116366.043226,4805.54966544
bathrooms,None,-77972.3305052,7565.0599109
lat,None,625433.835007,13058.3530975
long,None,-203958.603056,13268.1283731
bed_bath_rooms,None,26961.6249074,1956.36561554


In [47]:
model_3_features_model.get('coefficients')

name,index,value,stderr
(intercept),None,-52974974.0925,1615194.9441
sqft_living,None,529.196420557,7.69913498506
bedrooms,None,28948.527745,9395.72889101
bathrooms,None,65661.2072376,10795.3380702
lat,None,704762.148582,787360300.154
long,None,-137780.020094,787360300.123
bed_bath_rooms,None,-8478.36410611,2858.95391255
bedrooms_squared,None,-6072.38466234,1494.97042776
log_sqft_living,None,-563467.784241,17567.8230812
lat_plus_long,None,-83217.197992,787360300.087


**Quiz Question: What is the sign (positive or negative) for the coefficient/weight for 'bathrooms' in model 1?**

**Quiz Question: What is the sign (positive or negative) for the coefficient/weight for 'bathrooms' in model 2?**

Think about what this means.

# Comparing multiple models

Now that you've learned three models and extracted the model weights we want to evaluate which model is best.

First use your functions from earlier to compute the RSS on TRAINING Data for each of the three models.

In [49]:
# Compute the RSS on TRAINING data for each of the three models and record the values:
rss_model_1_features_train = get_residual_sum_of_squares(model_1_features_model, train_data, train_data['price'])
rss_model_2_features_train = get_residual_sum_of_squares(model_2_features_model, train_data, train_data['price'])
rss_model_3_features_train = get_residual_sum_of_squares(model_3_features_model, train_data, train_data['price'])
print rss_model_1_features_train, rss_model_2_features_train, rss_model_3_features_train

9.71328233536e+14 9.61592067848e+14 9.05276314543e+14


**Quiz Question: Which model (1, 2 or 3) has lowest RSS on TRAINING Data?** Is this what you expected?

Now compute the RSS on on TEST data for each of the three models.

In [50]:
# Compute the RSS on TESTING data for each of the three models and record the values:
rss_model_1_features_test = get_residual_sum_of_squares(model_1_features_model, test_data, test_data['price'])
rss_model_2_features_test = get_residual_sum_of_squares(model_2_features_model, test_data, test_data['price'])
rss_model_3_features_test = get_residual_sum_of_squares(model_3_features_model, test_data, test_data['price'])
print rss_model_1_features_test, rss_model_2_features_test, rss_model_3_features_test

2.26568089091e+14 2.24368799992e+14 2.51829318966e+14


**Quiz Question: Which model (1, 2 or 3) has lowest RSS on TESTING Data?** Is this what you expected?Think about the features that were added to each model from the previous.